In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

from keras.datasets import mnist
from keras.utils import to_categorical

import keras
from keras import layers
from keras import models
import tensorflow as tf

from tensorflow.keras.applications import VGG16

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

val_images = train_images[:6000]
val_labels = train_labels[:6000]

train_images = train_images[6000:]
train_labels = train_labels[6000:]


In [3]:
def procesa_imagenes(dataset):
  new_dataset = []
  for img in dataset:
    img = cv.resize(img, (56, 56))
    img = cv.cvtColor (img, cv.COLOR_GRAY2BGR)
    new_dataset.append(img)
  return np.array(new_dataset)

In [4]:
train_images = procesa_imagenes(train_images)
test_images = procesa_imagenes(test_images)
val_images = procesa_imagenes(val_images)

In [5]:
train_images = train_images.astype("float32") / 255
test_images = test_images.astype("float32") / 255
val_images = val_images.astype("float32") / 255

In [6]:
train_images.shape

(54000, 56, 56, 3)

In [7]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
val_labels = to_categorical(val_labels)

In [8]:
train_labels.shape

(54000, 10)

In [9]:
pre_trained_model = VGG16(input_shape=(56,56,3),
                          weights="imagenet",
                          include_top=False)

In [10]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [11]:
pre_trained_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 56, 56, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 56, 56, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 14, 14, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 14, 14, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 14, 14, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 7, 7, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 7, 7, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 7, 7, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [12]:
model = tf.keras.models.Sequential([
    pre_trained_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 1, 1, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,848,586 (56.64 MB)

 Trainable params: 133,898 (523.04 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [13]:
model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [14]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("model/my_keras_model.keras", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(train_images, train_labels,
                    batch_size=100, epochs=5,
                    validation_data=(val_images, val_labels),
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/5
540/540 ━━━━━━━━━━━━━━━━━━━━ 264s 488ms/step - accuracy: 0.5352 - loss: 1.8280 - val_accuracy: 0.8095 - val_loss: 0.9872
Epoch 2/5
540/540 ━━━━━━━━━━━━━━━━━━━━ 261s 484ms/step - accuracy: 0.8136 - loss: 0.8828 - val_accuracy: 0.8433 - val_loss: 0.6561
Epoch 3/5
540/540 ━━━━━━━━━━━━━━━━━━━━ 271s 503ms/step - accuracy: 0.8460 - loss: 0.6269 - val_accuracy: 0.8650 - val_loss: 0.5197
Epoch 4/5
540/540 ━━━━━━━━━━━━━━━━━━━━ 285s 528ms/step - accuracy: 0.8661 - loss: 0.5041 - val_accuracy: 0.8818 - val_loss: 0.4437
Epoch 5/5
540/540 ━━━━━━━━━━━━━━━━━━━━ 283s 524ms/step - accuracy: 0.8847 - loss: 0.4294 - val_accuracy: 0.8928 - val_loss: 0.3923
